In [1]:
import torch_linear_regression as tlr

import torch
import numpy as np
import sklearn
import sklearn.datasets
import matplotlib.pyplot as plt

# Basic usage

In [2]:
## Generate data for regression
X, Y = sklearn.datasets.make_regression(
    n_samples=100,
    n_features=2,
    n_informative=10,
    bias=2,
    noise=50,
    random_state=42,
)

In [3]:
## Create model
model_ols = tlr.OLS()
## Fit model
model_ols.fit(X=X, y=Y)
## Predict
Y_pred = model_ols.predict(X)
## Score model
score = model_ols.score(X=X, y=Y)
print(f"R^2: {score}")

R^2: 0.7819072511229945


## Prefitting

In [4]:
## Create model
model_ols_pf = tlr.OLS(prefit_X=X)
## Fit model
model_ols_pf.fit(X=X, y=Y)  ## You should still pass X again even though it is prefit
## Predict
Y_pred = model_ols_pf.predict(X)
## Check that coefficients are the same
if np.allclose(model_ols.coef_, model_ols_pf.coef_):
    print("All good, prefit model coefficients are the same as the model coefficients!")
else:
    print("Uh oh, prefit model coefficients are different from the model coefficients")

All good, prefit model coefficients are the same as the model coefficients!


Prefitting is useful when you want to fit some `X` data to many different `y` data. By passing the `X` matrix into the `prefit` argument in the initialization you can avoid recomputing the same thing over and over again.

In [5]:
import time

## Run a speed test

## Generate data for regression
X, Y = sklearn.datasets.make_regression(
    n_samples=1000,
    n_features=200,
    n_informative=100,
    n_targets=100,
    bias=2,
    noise=50,
    random_state=42,
)

## Fit without prefit
tic_multi = time.time()
model_ols_multi = tlr.OLS()
for y_i in Y.T:
    model_ols_multi.fit(X=X, y=y_i)
toc_multi = time.time()

## Fit with prefit
tic_multi_pf = time.time()
model_ols_multi_pf = tlr.OLS(prefit_X=X)
for y_i in Y.T:
    model_ols_multi_pf.fit(X=X, y=y_i)
toc_multi_pf = time.time()

## Check times
print(f"Time taken without prefit: {toc_multi - tic_multi}")
print(f"Time taken with prefit: {toc_multi_pf - tic_multi_pf}")

Time taken without prefit: 3.5273141860961914
Time taken with prefit: 0.10075020790100098


## Use numpy or torch arrays as inputs

In [6]:
## Make numpy and torch models
model_ols_np = tlr.OLS()
model_ols_np.fit(X=np.array(X), y=np.array(Y))

model_ols_torch = tlr.OLS()
model_ols_torch.fit(X=torch.as_tensor(X), y=torch.as_tensor(Y))

## Check that coefficients are the same
if np.allclose(model_ols_np.coef_, model_ols_torch.coef_.numpy()):
    print("All good, numpy model coefficients are the same as the torch model coefficients!")
else:
    print("Uh oh, numpy model coefficients are different from the torch model coefficients")

All good, numpy model coefficients are the same as the torch model coefficients!


## Run on a GPU

In [7]:
## Get device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

## Create model
model_ols_gpu = tlr.OLS()
model_ols_gpu.fit(X=torch.as_tensor(X, device=device), y=torch.as_tensor(Y, device=device))

## Check that coefficients are on the same device
print(f"Model coefficients device: {model_ols_gpu.coef_.device}")


Device: cuda
Model coefficients device: cuda:0


## Sklearn and PyTorch compatibility
The models are subclassed from `sklearn.base.BaseEstimator` and `torch.nn.Module` so they can be used in both frameworks.

##### From sklearn's `BaseEstimator`:

Methods:
- `fit`
- `predict`
- `fit_predict`
- `score`
  
Attributes:
- `coef_`
- `intercept_`

##### From PyTorch's `torch.nn.Module`:
- `forward`

Since the model is just matrix multiplication, addition, and possibly an SVD, you can treat the models' `fit` and `predict` methods as layers in a network and use them in a PyTorch pipeline.

In [8]:
## Make a new model
model_ols_2 = tlr.OLS()
model_ols_2.fit(X=X, y=Y)

## Predict using the `forward` method
## All three methods should give the same exact result
Y_pred_predict = model_ols_2.predict(X)
Y_pred_call = model_ols_2(X)
Y_pred_forward = model_ols_2.forward(X)

assert np.allclose(Y_pred_predict, Y_pred_call)
assert np.allclose(Y_pred_predict, Y_pred_forward)

In [336]:
## Let's make an OLS model without an intercept and use backpropagation to fit an intercept parameter
## Generate data for regression
X, Y = sklearn.datasets.make_regression(
    n_samples=100,
    n_features=2,
    noise=50,
    random_state=42,
)

## Add a known bias to the data
bias = 500  ## Set the bias to a known value. The param_intercept should converge to this value
X = torch.as_tensor(X)
Y = torch.as_tensor(Y) + bias

## Create model with fit_intercept=False
model_ols_noInt = tlr.OLS(prefit_X=X, fit_intercept=False)
## Create a parameter for the intercept that we will optimize
param_intercept = torch.nn.Parameter(torch.ones(1) * 1, requires_grad=True)

## Fit the model
fn_loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD([param_intercept], lr=1e-1)
losses = []
vals = []
for i in range(100):
    optimizer.zero_grad()
    Y_pred = model_ols_noInt.fit_predict(X=X, y=Y - param_intercept)
    # Y_pred = X @ (torch.linalg.inv(X.T @ X) @ X.T @ (Y - param_intercept))  ## This is equivalent to the line above
    loss = fn_loss(Y_pred, Y - param_intercept)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    vals.append(param_intercept.item())

## Print the final value of the intercept parameter
print(f"Final intercept parameter: {vals[-1]}")
print(f"True intercept parameter: {bias}")

Final intercept parameter: 501.08172607421875
True intercept parameter: 500


## Available models:

In [ ]:
tlr.OLS(
    fit_intercept=True,
    prefit_X=None,
)
tlr.Ridge(
    alpha=1e4,
    fit_intercept=True,
    prefit_X=None,
)
tlr.ReducedRankRidgeRegression(
    rank=5,
    alpha=1e4,
    fit_intercept=True,
    prefit_X=None,
)

enjoy